In [1]:
import os, sys
import itertools, random
import numpy as np

In [2]:
H = np.array([
    [3, 7, 0],
    [1, 5, 1],
    [0, 0, 1],
])

np.linalg.det(H)

8.0000000000000018

In [3]:
X = np.array([
    [1, 2, 1],
    [0.4, 0.5, 1],
    [-3, 4, 1],
    [9, 0.2, 1],
    [3, 2, 1],
    [4, 1.8, 1]
]).T

Y = np.dot(H, X)

print(X.shape, Y.shape)

(3, 6) (3, 6)


In [4]:
def homography_kron(x, y):
    Y = np.array([
        [0, -y[2], y[1]],
        [-y[2], 0, y[0]],
        [-y[1], y[0], 0]
    ])
    return np.kron(Y, x.T)

A = homography_kron(X[:, 0], Y[:, 0])
print(A.shape)
print(np.dot(A, H.ravel()))


(3, 9)
[ 0.  0.  0.]


In [5]:
def homography_kron_stack(X, Y):
    return np.vstack([homography_kron(x, y) for x,y in zip(X.T, Y.T)])

A = homography_kron_stack(X[:, :4], Y[:, :4])
print(A.shape)

(12, 9)


In [6]:
U,s,V = np.linalg.svd(A)
S = np.zeros_like(A)
S[:len(s), :len(s)] = np.diag(s)
np.allclose(A, np.dot(U, np.dot(S, V)))

True

In [7]:
V.T[:, -1].reshape(-1, X.shape[0])

array([[  3.23498320e-01,   7.54829412e-01,  -5.82875762e-14],
       [  1.07832773e-01,   5.39163866e-01,   1.07832773e-01],
       [  1.20736754e-15,   2.90826391e-15,   1.07832773e-01]])

In [8]:
H_DLT = V.T[:, -1].reshape(-1, X.shape[0]) / V[-1,-1]

In [9]:
np.allclose(H, H_DLT)

True

In [10]:
def dlt(X, Y):
    choices = np.random.permutation(X.shape[1])[:4]
    A = np.vstack([homography_kron(x, y) for x,y in zip(X[:, choices].T, Y[:, choices].T)])
    V = np.linalg.svd(A)[2].T
    H = V[:, -1].reshape(-1, X.shape[0]) / V[-1,-1]
    return H
    
dlt(X, Y)

array([[  3.00000000e+00,   7.00000000e+00,  -5.45097455e-11],
       [  1.00000000e+00,   5.00000000e+00,   1.00000000e+00],
       [  3.33712124e-13,   1.04796108e-12,   1.00000000e+00]])